In [1]:
import numpy as np
import pandas as pd

In [2]:
networks = ("human", "fly", "mouse", "rat", "bakers")
folder   = "../data/intact_output/"
dfs = {}
for i, netn in enumerate(networks):
    net = f"{folder}/{netn}.tsv"
    dfs[netn]  = pd.read_csv(net, sep = "\t")

/cluster/tufts/cowenlab/.envs/denoise/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
dfs["human"]

,ProteinA,ProteinB,Detection Method,Interaction Type,Confidence
0,uniprotkb:Q13643,uniprotkb:Q14192,"psi-mi:""MI:0018""(two hybrid)","psi-mi:""MI:0915""(physical association)",0.45
1,uniprotkb:P42566,uniprotkb:P42566,"psi-mi:""MI:0030""(cross-linking study)","psi-mi:""MI:0915""(physical association)",0.64
2,uniprotkb:P42566,uniprotkb:P42566,"psi-mi:""MI:0030""(cross-linking study)","psi-mi:""MI:0915""(physical association)",0.64
3,uniprotkb:P33993,uniprotkb:Q14192,"psi-mi:""MI:0018""(two hybrid)","psi-mi:""MI:0915""(physical association)",0.37
4,uniprotkb:P10275,uniprotkb:Q14192,"psi-mi:""MI:0018""(two hybrid)","psi-mi:""MI:0915""(physical association)",0.51
...,...,...,...,...,...
477327,uniprotkb:P38398,uniprotkb:P38398,"psi-mi:""MI:0007""(anti tag coimmunoprecipitation)","psi-mi:""MI:0914""(association)",0.65
477328,uniprotkb:P38398,uniprotkb:Q96RL1,"psi-mi:""MI:0007""(anti tag coimmunoprecipitation)","psi-mi:""MI:0914""(association)",0.78
477329,uniprotkb:Q96SB3,uniprotkb:P38398,"psi-mi:""MI:0007""(anti tag coimmunoprecipitation)","psi-mi:""MI:0915""(physical association)",0.5
477330,uniprotkb:Q13015,uniprotkb:Q7L0X0,"psi-mi:""MI:0006""(anti bait coimmunoprecipitation)","psi-mi:""MI:0915""(physical association)",0.4


In [10]:
import networkx as nx
Gs = {}
dinfo = []
for i, netn in enumerate(networks):
    Gs[netn] = nx.from_pandas_edgelist(dfs[netn], 
                                       "ProteinA", 
                                       "ProteinB",
                                      "Confidence")
    nnodes = Gs[netn].number_of_nodes()
    nedges = Gs[netn].number_of_edges()
    avdeg = np.average([x[1] for x in Gs[netn].degree()])
    aclust = nx.average_clustering(Gs[netn])
    dinfo.append((netn, nnodes, nedges, avdeg, aclust))
dfinfo = pd.DataFrame(dinfo, columns = ["network",
                                       "#nodes",
                                       "#edges",
                                       "average degree",
                                       "average clustering coefficient"])
dinfo

[('human', 32431, 301529, 18.595109617341432, 0.06985735022106601),
 ('fly', 11247, 42555, 7.567351293678314, 0.0274768933979802),
 ('mouse', 13967, 36541, 5.232476551872271, 0.0516698875238952),
 ('rat', 10792, 23315, 4.320793180133432, 0.07534761018842755),
 ('bakers', 6478, 70638, 21.808582895955542, 0.28587615896432006)]

In [12]:
dfinfo.to_latex("info.txt", index = None)

In [52]:
import json
for k in networks:
    dfs[k].loc[:, "ProteinA"] = dfs[k].loc[:, "ProteinA"].apply(lambda x: x.split(":")[1])
    dfs[k].loc[:, "ProteinB"] = dfs[k].loc[:, "ProteinB"].apply(lambda x: x.split(":")[1])
    #dfs[k].iloc[:, [0, 1]].to_csv(f"{k}.s.tsv", sep = "\t", header = None, index = None)
    nodemap = {k: i+1 for i, k in enumerate(set(dfs[k]["ProteinA"]).union(dfs[k]["ProteinB"]))}
    print("nodemap created")
    dfs[k].loc[:, "ProteinA"] = dfs[k].loc[:, "ProteinA"].apply(lambda x: nodemap[x])
    dfs[k].loc[:, "ProteinB"] = dfs[k].loc[:, "ProteinB"].apply(lambda x: nodemap[x])
    dfs[k] = dfs[k].drop_duplicates(subset=["ProteinA", "ProteinB"])
    print("annotated")
    dfs[k].iloc[:, [0, 1]].to_csv(f"../data/intact_output/working/{k}.a.tsv", sep = "\t", header = None, index = None)
    with open(f"../data/intact_output/working/{k}.json", "w") as fa:
        json.dump(nodemap, fa)

nodemap created
annotated
nodemap created
annotated
nodemap created
annotated
nodemap created
annotated
nodemap created
annotated


In [36]:
nodemaps = {}
pairsdf = {}
for i in range(len(networks)):
    with open(f"{folder}/working/{networks[i]}.json", "r") as  nj:
        nodemaps[networks[i]] = json.load(nj)
    for j in range(i):
        pairf = f"{folder}/{networks[i]}-{networks[j]}.tsv"
        pairsdf[(networks[i], networks[j])] = pd.read_csv(pairf, sep = "\t", header = None)

In [37]:
for key, df in pairsdf.items():
    p1, p2 = key
    map1 = nodemaps[p1]
    map2 = nodemaps[p2]
    df.loc[:, 0] = df.loc[:, 0].apply(lambda x: map2[x])
    df.loc[:, 1] = df.loc[:, 1].apply(lambda x: map1[x])
    maxdf = np.max(df[2])
    df.loc[:, 2] = df.loc[:, 2].apply(lambda x: x / maxdf)
    df.to_csv(f"{folder}/working/{p2}-{p1}.tsv", sep = "\t", header = None, index = None)
    print(df)

           0     1         2
0      12441   709  0.060855
1       3454   709  0.064251
2      14644   709  0.064393
3      29102   709  0.026465
4      27952   709  0.067506
...      ...   ...       ...
70427  12500  8213  0.015851
70428  24568  8213  0.010190
70429  17799  8213  0.040928
70430  20817  8213  0.013077
70431   7179  8213  0.009808

[70432 rows x 3 columns]
            0      1         2
0       31921  12601  0.028100
1       25633  12601  0.003021
2       26158  12601  0.002416
3       25006   8415  0.037262
4        3889   8415  0.002200
...       ...    ...       ...
359995  31358   3213  0.001328
359996  17923   3213  0.001349
359997  22369   3213  0.001147
359998  22916   3213  0.001361
359999   7130   3213  0.000993

[360000 rows x 3 columns]
           0      1         2
0      10530   1605  0.010343
1      10418   9829  0.015212
2       4762  12046  0.031134
3      10853   1703  0.015070
4       7927    553  0.048621
...      ...    ...       ...
46307   1448   54

In [46]:
df = pd.read_csv("../data/intact_output/working/human.a.tsv", sep = "\t", header = None)
df

,0,1
0,9726,19662
1,4765,4765
2,4765,4765
3,21300,19662
4,24403,19662
...,...,...
477327,23624,23624
477328,23624,18155
477329,15220,23624
477330,11872,12319


In [48]:
dfg =df.groupby(by = [0, 1], as_index=False).size()

,0,1,size
0,4,115,1
210359,20090,24523,1
210360,20090,24657,1
210363,20090,25767,1
210366,20090,26035,1
...,...,...,...
296128,28388,4178,30
118548,11533,25306,34
305467,29202,29202,35
313985,30085,26778,36
